In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

2023-07-26 09:51:05.091984: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 09:51:05.093304: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 09:51:05.121845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 09:51:05.122745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 09:51:05.794142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
fake_df = pd.read_csv('archive/Fake.csv')
true_df = pd.read_csv('archive/True.csv')

In [3]:
fake_df["category"] = 'fake'
true_df["category"] = 'real'

In [4]:
fake_df_downsampled = fake_df.sample(true_df.shape[0])
fake_df_downsampled.shape

(21417, 5)

In [5]:
combined_df = pd.concat([fake_df_downsampled, true_df])
combined_df

,title,text,subject,date,category
7911,Americans Demand Obama Pick Next Justice: GOP...,Ever since the passing of Supreme Court Justi...,News,"February 23, 2016",fake
3723,Trump ERUPTS At Secret Meeting With The Press...,Donald Trump has had a rocky relationship with...,News,"November 21, 2016",fake
5716,Hillary Gives HUGE Middle Finger To GOP Bigot...,While Donald the Trump is busy making the roun...,News,"June 26, 2016",fake
4397,"Trump LOSES IT, Has Complete Nervous Breakdow...","Trump has been having a very, very bad week. F...",News,"October 2, 2016",fake
13091,OBAMA DEFENDS KAEPERNICK’S Decision to Disresp...,When Obama had the opportunity to speak out ag...,politics,"Sep 5, 2016",fake
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",real
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",real
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",real
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",real


In [6]:
combined_df.dropna(subset=['title', 'text'], inplace=True)
combined_df.isna().sum()

title       0
text        0
subject     0
date        0
category    0
dtype: int64

In [7]:
combined_df.drop_duplicates(inplace=True)
combined_df.drop_duplicates(subset='title', inplace=True)
combined_df.drop_duplicates(subset='text', inplace=True)

In [8]:
combined_df['input_text'] = combined_df['title'] + " " + combined_df['text']

In [10]:
X = combined_df[['input_text']].values
y = combined_df[['category']].values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34, stratify=y)

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [14]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

/home/saadsameerkhan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
model.fit(X_train, y_train, epochs=1)

Epoch 1/10
928/928 [==============================] - 5013s 5s/step - loss: 0.3262 - accuracy: 0.8842 - precision: 0.8753 - recall: 0.9254
Epoch 2/10
  2/928 [..............................] - ETA: 1:24:31 - loss: 0.1395 - accuracy: 0.9688 - precision: 0.9744 - recall: 0.9744

KeyboardInterrupt: 

In [20]:
y_test = label_encoder.fit_transform(y_test)
model.evaluate(X_test, y_test)

/home/saadsameerkhan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


232/232 [==============================] - 1249s 5s/step - loss: 0.2016 - accuracy: 0.9360 - precision: 0.9335 - recall: 0.9539


[0.20161817967891693,
 0.9360010623931885,
 0.9334900379180908,
 0.9538905024528503]

In [27]:
fake_1 = combined_df[combined_df["category"] == "fake"]

In [28]:
fake_1

,title,text,subject,date,category,input_text
7911,Americans Demand Obama Pick Next Justice: GOP...,Ever since the passing of Supreme Court Justi...,News,"February 23, 2016",fake,Americans Demand Obama Pick Next Justice: GOP...
3723,Trump ERUPTS At Secret Meeting With The Press...,Donald Trump has had a rocky relationship with...,News,"November 21, 2016",fake,Trump ERUPTS At Secret Meeting With The Press...
5716,Hillary Gives HUGE Middle Finger To GOP Bigot...,While Donald the Trump is busy making the roun...,News,"June 26, 2016",fake,Hillary Gives HUGE Middle Finger To GOP Bigot...
4397,"Trump LOSES IT, Has Complete Nervous Breakdow...","Trump has been having a very, very bad week. F...",News,"October 2, 2016",fake,"Trump LOSES IT, Has Complete Nervous Breakdow..."
13091,OBAMA DEFENDS KAEPERNICK’S Decision to Disresp...,When Obama had the opportunity to speak out ag...,politics,"Sep 5, 2016",fake,OBAMA DEFENDS KAEPERNICK’S Decision to Disresp...
...,...,...,...,...,...,...
16979,EVIL GEORGE SOROS vs HUNGARIAN PM: The Battle ...,George Soros gets ugly with the name calling t...,Government News,"Nov 6, 2015",fake,EVIL GEORGE SOROS vs HUNGARIAN PM: The Battle ...
7717,Three Liberal Female Supreme Court Justices S...,The United States Supreme Court is currently d...,News,"March 3, 2016",fake,Three Liberal Female Supreme Court Justices S...
3875,WATCH: Van Jones Says EXACTLY What Needs To B...,The election result tonight is hard to swallow...,News,"November 9, 2016",fake,WATCH: Van Jones Says EXACTLY What Needs To B...
6211,"Supreme Court Gives A Big ‘F You’ To GOP, Rej...",The Supreme Court did something amazing on Mon...,News,"May 23, 2016",fake,"Supreme Court Gives A Big ‘F You’ To GOP, Rej..."


In [29]:
real_1 = combined_df[combined_df["category"] == "real"]

In [31]:
fake_var = fake_1.iloc[0, 5]
fake_var

' Americans Demand Obama Pick Next Justice: GOP Gives Them A Huge Middle Finger (VIDEO)  Ever since the passing of Supreme Court Justice Antonin Scalia, Republicans have insisted on turning the constitutionally-backed nomination process into a political sideshow to deprive Obama of his legal right to name a new Justice and to deprive Americans of their say in the matter.Senate Republicans, led by their Obstructor-in-Chief Mitch McConnell, have repeatedly made the astounding claim that they will refuse to hold nomination hearings for any person Obama may choose. Sight unseen. Instead, they will spend the next 340 days with an empty seat on the Supreme Court   a glowing middle finger neon sign built in the way of the democratic process.For their part, the American people have made it absolutely clear that this sort of political point scoring is inappropriate, offensive, and unwanted. When Fox News (Fox News!) ran a poll asking if Americans supported Obama naming a new justice, an overwhe

In [32]:
fake_var_2 = fake_1.iloc[1, 5]
fake_var_2

' Trump ERUPTS At Secret Meeting With The Press, Transition In Chaos (TWEETS) Donald Trump has had a rocky relationship with the press ever since his presidential campaign, and he s only punished them further for trying to report the truth since becoming the President-elect.On Monday, Trump was set to have an off the record meeting with television networks to try and hash out presidential press access. This meeting was particularly important because Trump has previously limited press access, and has ditched two protocol press pools since getting elected.Some reporters in attendance were NBC s Deborah Turness, Lester Holt and Chuck Todd; ABC s James Goldston, George Stephanopoulos, David Muir and Martha Raddatz; CBS  Norah O Donnell and Charlie Rose; Fox News  Bill Shine, Jack Abernethy, Jay Wallace, Suzanne Scott; MSNBC s Phil Griffin; and CNN s Jeff Zucker and Erin Burnett. It was a good group, and it could have been a very effective meeting if only things had gone according to plan.U

In [33]:
fake_var_3 = fake_1.iloc[2, 5]
fake_var_3

' Hillary Gives HUGE Middle Finger To GOP Bigots With Surprise Showing In NYC Pride Parade While Donald the Trump is busy making the rounds decrying immigration, Hillary Clinton decided to make a surprise appearance in New York City for Pride. She joined NYC mayor Bill de Blasio and Governor Andrew Cuomo on the parade route near the Stonewall Inn   an especially fitting show of solidarity and support following the Orlando massacre.Hillary has worked hard on reaching out to the LGBT community, and right now, they need it more than ever. The entire community was rocked to its core by the Orlando massacre, and how they feel right now is evident in Pride celebrations that are more subdued than usual.Some have avoided Pride altogether out of fear. Others have refused to engage anybody that might show the slightest hint that they re bigoted towards the LGBT community. Make no mistake, Orlando was a direct attack on their community and they have every reason to be terrified right now.Where ar

In [34]:
real_var_1 = real_1.iloc[1, 5]
real_var_2 = real_1.iloc[2, 5]
real_var_3 = real_1.iloc[3, 5]

In [35]:
real_var_1

'U.S. military to accept transgender recruits on Monday: Pentagon WASHINGTON (Reuters) - Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District 

In [36]:
real_var_2

"Senior U.S. Republican senator: 'Let Mr. Mueller do his job' WASHINGTON (Reuters) - The special counsel investigation of links between Russia and President Trump’s 2016 election campaign should continue without interference in 2018, despite calls from some Trump administration allies and Republican lawmakers to shut it down, a prominent Republican senator said on Sunday. Lindsey Graham, who serves on the Senate armed forces and judiciary committees, said Department of Justice Special Counsel Robert Mueller needs to carry on with his Russia investigation without political interference. “This investigation will go forward. It will be an investigation conducted without political influence,” Graham said on CBS’s Face the Nation news program. “And we all need to let Mr. Mueller do his job. I think he’s the right guy at the right time.”  The question of how Russia may have interfered in the election, and how Trump’s campaign may have had links with or co-ordinated any such effort, has loome

In [37]:
real_var_3

'FBI Russia probe helped by Australian diplomat tip-off: NYT WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBI’s decision to open a counter-intelligence investigation of Moscow’s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American counterparts when leaked Democratic emails began appearing online, according to the newspaper, which cited four current and former U.S. and foreign officials. Besides the information from the Australians, the probe by the Federal Bureau of Investigation was also propelled by intelligence from other friendly governments, including the British a

In [38]:
fake2 = "World's Largest Diamond Unearthed in Africa! Miners in Africa have made an astonishing find, unearthing the world's largest diamond ever recorded. The massive gemstone, weighing over 3,000 carats, is estimated to be worth billions of dollars. The diamond's flawless clarity and stunning size have captivated the jewelry industry, with top luxury brands vying for the chance to own this historic gem. The discovery is expected to have a significant impact on the global diamond market, and experts predict it will become a priceless centerpiece in the crown jewels of a royal family."
fake1 = "NASA Announces Discovery of Alien Life on Mars! NASA scientists have made a groundbreaking announcement, revealing the discovery of alien life on Mars. The evidence comes from recent data collected by the Curiosity rover, which indicates the presence of microbial life on the red planet. This discovery marks a historic moment in space exploration and opens up new possibilities for the existence of extraterrestrial life in our solar system. The scientific community is abuzz with excitement, and the search for more evidence of life on Mars is now a top priority for NASA's future missions."

In [39]:
real1 = "U.S. Military to Accept Transgender Recruits on Monday: Pentagon Transgender people will be allowed to enlist in the U.S. military starting Monday, as ordered by federal courts. The Pentagon stated that it will not appeal rulings that blocked President Trump's transgender ban. The decision comes after federal appeals courts rejected the administration's request to delay accepting transgender recruits. The move marks a significant step toward equality in the military and follows multiple inquiries into Russia's interference in the 2016 election and possible links to the Trump campaign."
real2 = "FBI Russia Probe Helped by Australian Diplomat Tip-Off: NYT The FBI's investigation into Russia's contacts with the Trump campaign was propelled by intelligence from Australian diplomat Alexander Downer. George Papadopoulos, a former Trump campaign adviser, told Downer in May 2016 that Russia had political dirt on Hillary Clinton. Australian officials later passed this information to their American counterparts. Papadopoulos pleaded guilty to lying to FBI agents about his contacts with individuals claiming ties to Russian officials. Special Counsel Robert Mueller's investigation continues to be a topic of interest and debate."

In [40]:
list1 = [fake1, fake2, real1, real2]
list1

["NASA Announces Discovery of Alien Life on Mars! NASA scientists have made a groundbreaking announcement, revealing the discovery of alien life on Mars. The evidence comes from recent data collected by the Curiosity rover, which indicates the presence of microbial life on the red planet. This discovery marks a historic moment in space exploration and opens up new possibilities for the existence of extraterrestrial life in our solar system. The scientific community is abuzz with excitement, and the search for more evidence of life on Mars is now a top priority for NASA's future missions.",
 "World's Largest Diamond Unearthed in Africa! Miners in Africa have made an astonishing find, unearthing the world's largest diamond ever recorded. The massive gemstone, weighing over 3,000 carats, is estimated to be worth billions of dollars. The diamond's flawless clarity and stunning size have captivated the jewelry industry, with top luxury brands vying for the chance to own this historic gem. T

In [41]:
model.predict(list1)

1/1 [==============================] - 1s 1s/step


array([[0.23849733],
       [0.48847654],
       [0.9264428 ],
       [0.9278714 ]], dtype=float32)

In [44]:
cnn = '''Taiwan police investigated after botched sting operation leads to sexual assault of minor Two Taiwan police officers are under investigation after a botched sting operation led to a minor being sexually assaulted, prompting public uproar and an official apology from the local police department.

The incident began in November 2020 when an unnamed 13-year-old girl met a man, Chang Ming-hsin, on a dating app, according to a court verdict handed down earlier this month triggering an investigation into police conduct.

According to the court verdict, Chang solicited sex from the girl, offering 5,000 New Taiwan dollars (about $160), and asked her for nude photos. They agreed to meet at a hotel several days later – and when the girl didn’t show up, Chang threatened to leak her explicit photos online.

She then reported the case to the Zhongli Precinct of the Taoyuan Police Department, in Taoyuan city, located just outside Taipei, the verdict said. The police put together a sting operation, with the girl agreeing to meet with Chang under the pretense of having sex with him, the verdict revealed.

A spokesperson from the precinct said last week that the girl had been accompanied by a guardian when she reported the case, when police took her statement, and when the operation took place. The guardian had agreed to the sting operation, the spokesperson added.

During the operation, the girl met Chang outside a fast food restaurant and asked him to step out of the car, with two police officers monitoring the meet-up on scene. But he repeatedly demanded she get in instead, and drove the girl to a parking space about 200 to 300 meters away (about 656 to 984 feet).

Chang ordered her to perform oral sex on him and she complied out of fear, the verdict said.

Afterward, Chang drove the girl back to the restaurant, where police arrested him. His case reached a verdict on July 13, when the Taoyuan District Court sentenced Chang to eight years in prison on three sexual offense counts, including coercion of a minor to engage in obscene acts and transactional sex.

But the police involvement in the botched sting has garnered further international attention and outcry.

Focus Taiwan, an offshoot of Taiwan’s official Central News Agency, reported that the girl had been instructed by police not to enter Chang’s car, and that when she did, the police officers on scene had tried to run after the car – but they were “not fast enough” to stop it.

The girl reportedly had not mentioned the assault during the following investigation by prosecutors and police, only disclosing it during Chang’s court trial, according to Focus Taiwan.

But on Saturday, a spokesperson from the Taoyuan Police Department, which oversees the police precinct, released a video statement apologizing for negligence of the girl’s personal safety and the harm caused to her.

The two officers involved in the sting operation have been referred to the district prosecutor’s office for investigation of their alleged negligence and failure in performing their duties, said the spokesperson. In the meantime, they have received demerits and been transferred to only handle administrative police duties.

Two other police officers, who had oversight of the operation, have also received demerits, said the spokesperson, citing damage to the police department’s reputation.'''

In [46]:
list3 = [cnn]

In [47]:
model.predict(list3)

1/1 [==============================] - 0s 218ms/step


array([[0.90361345]], dtype=float32)

In [52]:
def fake_news_predictor(article):
    return_result = model.predict([article])
    if return_result[0] > 0.6:
        return "Real news"
    else:
        return "Fake news"

In [53]:
fake_news_predictor(cnn)

1/1 [==============================] - 0s 211ms/step


'Real news'

In [ ]:
model.save('trained_model_v1')